## Basic preprocessing
First of all, we need to drop duplicates from our SMILES dataset. Then we need to check compounds for chemistry problems, readability, size and delete inappropriate SMILES from dataframe.

In [ ]:
!pip install rdkit-pypi

     |████████████████████████████████| 22.5 MB 1.8 MB/s 


In [ ]:
import os
from collections import defaultdict
import pandas as pd

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import PandasTools

In [ ]:
smi_file = 'new_compounds.json'

smiles_column = 'smiles'
min_num_atoms_to_allow = 4
output_filename = 'data'

In [ ]:
import json


with open(smi_file, 'r') as f:
    smiles_json = json.load(f)
    smiles_list = []
    for compounds_list in smiles_json.values():
    	smiles_list.extend(compounds_list)
    
len(smiles_list)

836

In [ ]:
df = pd.DataFrame(smiles_list, columns=[smiles_column])
print(df.shape)
df.head()

(836, 1)


,smiles
0,Cc1ccc(C2=NNN(C[C@@H]3COc4ccccc4O3)[C@H]2C(=O)...
1,CN(C)CC1=NNN(C[C@@H]2COc3ccccc3O2)[C@@H]1c1cn[...
2,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(=C)C)cc1
3,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(=C)O)cc1
4,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(C)(C)O...


In [ ]:
dict_statistics = defaultdict(list)
dict_statistics['initial_size'].append(df.shape[0])
    
# Delete structures duplicated by SMILES
smiles_duplicated = df.duplicated(smiles_column)
dict_statistics['duplicated_by_smiles'].append(df[smiles_duplicated].shape[0])
df = df[~smiles_duplicated]

# Delete structures with non-readable SMILES
df['ROMol'] = df[smiles_column].apply(lambda x: Chem.MolFromSmiles(x))
non_readable_smi = df['ROMol'] == None
dict_statistics['non-readable_smiles'].append(df[non_readable_smi].shape[0])
df = df[~non_readable_smi]

# Delete structures with 'chemistry problems'
df['chemistry_problems'] = df[smiles_column].apply(lambda x: len(Chem.DetectChemistryProblems(Chem.MolFromSmiles(x, sanitize=True))))
chemistry_problems = df['chemistry_problems'] == 1
dict_statistics['chemistry_problems'].append(df[chemistry_problems].shape[0])
df = df[~chemistry_problems]
df = df.drop(['chemistry_problems'], axis=1)

# Delete extra-small compounds (currently: <= 4 atoms)
df['atoms_num'] = df['ROMol'].apply(lambda x: x.GetNumAtoms())
extra_small_molecules = df['atoms_num'] < min_num_atoms_to_allow
dict_statistics['less_than_' + str(min_num_atoms_to_allow) + '_atoms'].append(df[extra_small_molecules].shape[0])
df = df[~extra_small_molecules]
df = df.drop(['ROMol', 'atoms_num'], axis=1)

dict_statistics['filtered_size'].append(df.shape[0])

df_statistics = pd.DataFrame(dict_statistics)
df_statistics.to_csv(output_filename + '_preprocessed_statistics.csv', index=False)

In [ ]:
print(df.shape)
df.head()

(777, 2)


,smiles,id
0,Cc1ccc(C2=NNN(C[C@@H]3COc4ccccc4O3)[C@H]2C(=O)...,id-1
1,CN(C)CC1=NNN(C[C@@H]2COc3ccccc3O2)[C@@H]1c1cn[...,id-2
2,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(=C)C)cc1,id-3
3,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(=C)O)cc1,id-4
4,Cc1ccc(C(=O)CN2NN=C(CNC(C)(C)C)[C@@H]2C(C)(C)O...,id-5


## Generate IDs

In [ ]:
df = df.reset_index(drop=True).reset_index()
df['index'] = df['index'] + 1
df['id'] = 'id-' + df['index'].astype(str)
df = df.drop(['index'], axis=1)

In [ ]:
df.to_csv(output_filename + '_preprocessed.csv', index=False)

## Remove non-trusted SMILES
To convert SMILES to 3D structures, algorithm ETKDGv3 from RDKit was used. We can convert mols back to SMILES and check if compounds are equal to original ones. If not, compounds are non-trusted. 

After we did that, 765 compounds were marked as trusted.

## Convert to pdb
The input format in AutoDock Vina is pdbqt, so we need to convert our mols to pdb first, and then in pdbqt. For pdbqt conversion OpenBabelGUI software can be used

In [ ]:
import glob
import shutil
from tqdm import tqdm
from rdkit import Chem


# Directory for generated 3D MOLs
mol_directory = 'mol_3d_correct'

# Directory for RDKit's PDBs
pdb_directory = 'pdb'

format_in = 'mol'
format_out = 'pdb'

# Create directories for PDBs
if not os.path.exists(pdb_directory):
    os.mkdir(pdb_directory)
else:
    print('path ' + str(pdb_directory) + ' already exists!')
    #shutil.rmtree(pdb_directory)

for mol_file in tqdm(glob.glob(mol_directory + os.sep + '*.' + format_in)):
    mol_filename = mol_file.split(os.sep)[-1].split('.')[0]
    mol = Chem.MolFromMolFile(mol_file, removeHs=False)
    Chem.MolToPDBFile(mol, pdb_directory + os.sep + mol_filename + '.' + format_out, flavor=20)


100%|██████████| 765/765 [00:00<00:00, 988.38it/s] 
